# On charge les données du dataset EDC


In [2]:
import duckdb
import pandas as pd
import os
import numpy as np
from pipelines.tasks.config.common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

query = """
SELECT edc_prelevements.dateprel, edc_prelevements.heureprel,edc_prelevements.inseecommuneprinc, edc_prelevements.nomcommuneprinc, edc_prelevements.cdreseau, edc_prelevements.cdreseauamont, edc_resultats.referenceprel, edc_resultats.valtraduite
FROM edc_prelevements
JOIN edc_resultats
ON edc_prelevements.referenceprel = edc_resultats.referenceprel
WHERE
(dateprel >= '2024-01-01')
"""

edc = con.sql(query)
edc = edc.df()

# On charge les données du dataset EDR


In [4]:
edr_udi_plv = pd.read_csv(
    r"C:\Users\Admin\Downloads\edr\UDI_PLV_202412.txt", delimiter=","
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16816\612874238.py:1: DtypeWarning: Columns (0,3,4,6,11,13,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  edr_udi_plv = pd.read_csv(


In [6]:
edr_udi_plv.columns

Index(['distrcode', 'distrlib', 'distrsiret', 'cddept', 'inseecommune',
       'nomcommune', 'cdreseau', 'nomreseau', 'codetypeinstallation',
       'nomtypeinstallation', 'inae', 'cdpointsurv', 'nompointsurv',
       'referenceprel', 'dateprel', 'heureprel', 'finaliteprel',
       'conclusionprel', 'cdtypeeau', 'libtypeeau', 'plvconformitebacterio',
       'plvconformitechimique', 'plvcomplet', 'plvcdorganismepayeur',
       'typevisite', 'plvcondition', 'preleveur', 'distrsiren',
       'plvconformiterefbacterio', 'plvconformiterefchimique', 'cddeptsandre',
       'dtautorisation', 'champcaptant', 'codebrgm', 'codebss', 'dmajplv',
       'dmodifplv', 'dmodifins', 'representativite', 'coord_x', 'coord_y'],
      dtype='object')

# Analyse des différences


In [11]:
query_prel = """
SELECT DISTINCT COUNT(referenceprel) AS Nb_prel
FROM edc_prelevements
"""

edc_prel = con.sql(query_prel)
edc_prel = edc_prel.df()

In [12]:
edc_prel

,Nb_prel
0,2083345


In [14]:
query_preledr = """
SELECT DISTINCT COUNT(referenceprel) AS Nb_prel
FROM edr_udi_plv
"""

edr_prel = con.sql(query_preledr)
edr_prel = edr_prel.df()

In [15]:
edr_prel

,Nb_prel
0,24532


On regarde combien de referenceprel de EDC sont dans EDR


La condition edc.cdreseauamont IS NULL permet de retirer les prélèvements hors UDI de la table EDC


In [20]:
query_comp1 = """
SELECT DISTINCT edc.referenceprel
FROM   edc
WHERE 
    (edc.referenceprel NOT IN (SELECT DISTINCT edr_udi_plv.referenceprel FROM edr_udi_plv))
    AND
    (edc.cdreseauamont IS NULL)
"""

comp1 = con.sql(query_comp1)
comp1 = comp1.df()

In [21]:
comp1

,referenceprel
0,09100195586
1,09100195590
2,09100195601
3,09100195625
4,09100195669
...,...
201139,08500470915
201140,05700169293
201141,06100143826
201142,02800124866


# Dans l'autre sens, on regarde les referenceprel présents dans EDR mais pas dans EDC


In [25]:
edr_udi_plv.columns

Index(['distrcode', 'distrlib', 'distrsiret', 'cddept', 'inseecommune',
       'nomcommune', 'cdreseau', 'nomreseau', 'codetypeinstallation',
       'nomtypeinstallation', 'inae', 'cdpointsurv', 'nompointsurv',
       'referenceprel', 'dateprel', 'heureprel', 'finaliteprel',
       'conclusionprel', 'cdtypeeau', 'libtypeeau', 'plvconformitebacterio',
       'plvconformitechimique', 'plvcomplet', 'plvcdorganismepayeur',
       'typevisite', 'plvcondition', 'preleveur', 'distrsiren',
       'plvconformiterefbacterio', 'plvconformiterefchimique', 'cddeptsandre',
       'dtautorisation', 'champcaptant', 'codebrgm', 'codebss', 'dmajplv',
       'dmodifplv', 'dmodifins', 'representativite', 'coord_x', 'coord_y'],
      dtype='object')

In [26]:
query_comp2 = """
SELECT DISTINCT edr_udi_plv.referenceprel, edr_udi_plv.inseecommune, edr_udi_plv.nomcommune, edr_udi_plv.cdreseau
FROM   edr_udi_plv
WHERE edr_udi_plv.referenceprel NOT IN (SELECT DISTINCT edc.referenceprel FROM edc)
"""

comp2 = con.sql(query_comp2)
comp2 = comp2.df()

In [27]:
comp2

,referenceprel,inseecommune,nomcommune,cdreseau
0,00900156705,09171,LORDAT,009001108
1,00900156702,09188,MERCUS-GARRABET,009001130
2,00900156524,09206,MONTFERRIER,009003767
3,00900156252,09210,MONTOULIEU,009001160
4,00900156870,09262,SAINT-JEAN-D'AIGUES-VIVES,009003201
...,...,...,...,...
8120,02200236769,22200,PLEVEN,022000284
8121,02200237135,22258,QUESSOY,022000995
8122,02200236918,22266,ROSTRENEN,022000624
8123,02300079694,23105,LAVAVEIX-LES-MINES,023000074


8125 prélevements sont dans EDR et pas dans EDC, on va chercher à comprendre d'où viennent ces prélèvements


On commence par vérifier s'il ne s'agit pas juste de prélèvements faits sur une commune dite principale et rapportés sur les communes qui dépendent de l'UDI située dans la commune principale


In [28]:
query_comp3 = """
SELECT DISTINCT GROUP_CONCAT(edr_udi_plv.referenceprel) AS Liste_referenceprel, GROUP_CONCAT(edr_udi_plv.inseecommune) AS Liste_inseecommune, GROUP_CONCAT(edr_udi_plv.nomcommune) AS nomcommune, edr_udi_plv.cdreseau
FROM   edr_udi_plv
WHERE edr_udi_plv.referenceprel NOT IN (SELECT DISTINCT edc.referenceprel FROM edc)
GROUP BY edr_udi_plv.cdreseau
"""

comp3 = con.sql(query_comp3)
comp3 = comp3.df()

In [33]:
comp3

,Liste_referenceprel,Liste_inseecommune,nomcommune,cdreseau
0,"6400150718,6400193067,6400214570,6400215403,64...","64445,64445,64445,64445,64445,64445,64445,6444...","PAU,PAU,PAU,PAU,PAU,PAU,PAU,PAU,PAU,PAU,PAU,PA...",64000432
1,6500154858,65344,OSSUN,65000659
2,6600213715,66058,CORNEILLA-LA-RIVIERE,66000194
3,6600213707,66108,MILLAS,66000326
4,6900167582,69171,SAIN-BEL,69000419
...,...,...,...,...
3423,8200088827,82044,CORBARIEU,82000170
3424,8500472267,85160,NESMY,85000181
3425,"8900133272,8900133274","89262,89262","MOLOSMES,MOLOSMES",89000571
3426,02900256761,29042,CROZON,029000914


In [57]:
edr_udi_plv.columns

Index(['distrcode', 'distrlib', 'distrsiret', 'cddept', 'inseecommune',
       'nomcommune', 'cdreseau', 'nomreseau', 'codetypeinstallation',
       'nomtypeinstallation', 'inae', 'cdpointsurv', 'nompointsurv',
       'referenceprel', 'dateprel', 'heureprel', 'finaliteprel',
       'conclusionprel', 'cdtypeeau', 'libtypeeau', 'plvconformitebacterio',
       'plvconformitechimique', 'plvcomplet', 'plvcdorganismepayeur',
       'typevisite', 'plvcondition', 'preleveur', 'distrsiren',
       'plvconformiterefbacterio', 'plvconformiterefchimique', 'cddeptsandre',
       'dtautorisation', 'champcaptant', 'codebrgm', 'codebss', 'dmajplv',
       'dmodifplv', 'dmodifins', 'representativite', 'coord_x', 'coord_y'],
      dtype='object')

In [65]:
comp2["referenceprel"].to_numpy()

array(['00900156705', '00900156702', '00900156524', ..., '02200236918',
       '02300079694', '02500148224'], shape=(8125,), dtype=object)

In [62]:
query_comp4 = """
SELECT referenceprel, inseecommune, nomcommune, cdreseau,dateprel, heureprel,nomtypeinstallation
FROM edr_udi_plv
WHERE referenceprel = '02900256761'
"""

comp4 = con.sql(query_comp4)
comp4 = comp4.df()

In [63]:
comp4

,referenceprel,inseecommune,nomcommune,cdreseau,dateprel,heureprel,nomtypeinstallation
0,02900256761,29042,CROZON,029000914,2024-11-26,09h22,UNITE DE DISTRIBUTION
